# Imports

In [25]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
covid_path = "../datasets/COVID19BE_CASES_MUNI.csv"
df = pd.read_csv(covid_path)

In [55]:
geoloc_path = "../datasets/postal_code_nis_disctric.csv"
df_geo = pd.read_csv(geoloc_path)

# Exploratory data analysis

In [8]:
df.head()

,NIS5,DATE,TX_DESCR_NL,TX_DESCR_FR,TX_ADM_DSTR_DESCR_NL,TX_ADM_DSTR_DESCR_FR,PROVINCE,REGION,CASES
0,11002.0,2020-03-01,Antwerpen,Anvers,Arrondissement Antwerpen,Arrondissement d’Anvers,Antwerpen,Flanders,<5
1,21004.0,2020-03-01,Brussel,Bruxelles,Arrondissement Brussel-Hoofdstad,Arrondissement de Bruxelles-Capitale,Brussels,Brussels,<5
2,21009.0,2020-03-01,Elsene,Ixelles,Arrondissement Brussel-Hoofdstad,Arrondissement de Bruxelles-Capitale,Brussels,Brussels,<5
3,21014.0,2020-03-01,Sint-Joost-ten-Node,Saint-Josse-ten-Noode,Arrondissement Brussel-Hoofdstad,Arrondissement de Bruxelles-Capitale,Brussels,Brussels,<5
4,23062.0,2020-03-01,Overijse,Overijse,Arrondissement Halle-Vilvoorde,Arrondissement de Hal-Vilvorde,VlaamsBrabant,Flanders,<5


In [21]:
df.shape

(91831, 5)

### Remove columns

We need covid cases per municipality. Many columns on this dataset are useless. Before any further analysis, we will remove them.

In [13]:
df.columns

Index(['NIS5', 'DATE', 'TX_DESCR_NL', 'TX_DESCR_FR', 'TX_ADM_DSTR_DESCR_NL',
       'TX_ADM_DSTR_DESCR_FR', 'PROVINCE', 'REGION', 'CASES'],
      dtype='object')

In [14]:
df.drop(columns=['TX_ADM_DSTR_DESCR_NL', 'TX_ADM_DSTR_DESCR_FR', 'PROVINCE', 'REGION'], inplace=True)

In [15]:
df.head(3)

,NIS5,DATE,TX_DESCR_NL,TX_DESCR_FR,CASES
0,11002.0,2020-03-01,Antwerpen,Anvers,<5
1,21004.0,2020-03-01,Brussel,Bruxelles,<5
2,21009.0,2020-03-01,Elsene,Ixelles,<5


### Missing values

This dataset seems to have some missing values. We will remove them.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91831 entries, 0 to 91830
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NIS5         91517 non-null  float64
 1   DATE         91767 non-null  object 
 2   TX_DESCR_NL  91517 non-null  object 
 3   TX_DESCR_FR  91517 non-null  object 
 4   CASES        91831 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.5+ MB


In [20]:
df.isna().sum()

NIS5           314
DATE            64
TX_DESCR_NL    314
TX_DESCR_FR    314
CASES            0
dtype: int64

Drop all "Na/NA/NaN"

In [24]:
df = df.dropna()

### Cases

Cases has a "<5" (less than five) value.

As it's not possible in my frametime to re-construct the dataset and get back real numbers, I'll just replace them by 1.

In [29]:
df['CASES'].value_counts()

<5     60072
5       4799
6       3484
7       2870
8       2341
       ...  
297        1
199        1
356        1
474        1
594        1
Name: CASES, Length: 323, dtype: int64

Replace "<5" by 1 and set this columns as int

In [32]:
df['CASES'].replace('<5', 1, inplace=True)

In [42]:
df['CASES'] = df['CASES'].astype(int)

<ipython-input-42-7f820b2eb13c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CASES'] = df['CASES'].astype(int)


In [41]:
df['CASES'].dtype

dtype('int64')

### Date

Later we'll get data by week. We need our 'Date' columns to becode a pandas datetime:

In [43]:
df['DATE'] = pd.to_datetime(df['DATE'])

<ipython-input-43-755015090936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE'] = pd.to_datetime(df['DATE'])


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91454 entries, 0 to 91765
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   NIS5         91454 non-null  float64       
 1   DATE         91454 non-null  datetime64[ns]
 2   TX_DESCR_NL  91454 non-null  object        
 3   TX_DESCR_FR  91454 non-null  object        
 4   CASES        91454 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 4.2+ MB


### NIS5

The "NIS5" or "INS code" is an unique identifier for each belgian municipality. It is useful to merge this dataset with localisation dataset.

In [45]:
df['NIS5'] = df['NIS5'].astype(int)

<ipython-input-45-a165034c2693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NIS5'] = df['NIS5'].astype(int)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91454 entries, 0 to 91765
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   NIS5         91454 non-null  int64         
 1   DATE         91454 non-null  datetime64[ns]
 2   TX_DESCR_NL  91454 non-null  object        
 3   TX_DESCR_FR  91454 non-null  object        
 4   CASES        91454 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.2+ MB


# Dataset transformation

To comply with the streamlit app, the data need to be transformed as the following:

 - Represent cases by weeks
 - Have geolocation data
 - Be reshaped as one dataframe per week, containing as many entries/location as cases/location.

### Group by week

We substract one week (7 days), as we want to sum for the week ahead of the date, not the week before that date

In [47]:
df['week_date'] = df['DATE'] - pd.to_timedelta(7, unit='d')

<ipython-input-47-6b47a9eb87ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week_date'] = df['DATE'] - pd.to_timedelta(7, unit='d')


In [51]:
df = df.groupby(['NIS5', pd.Grouper(key='DATE', freq='W-MON')])['CASES'].sum().reset_index().sort_values('DATE')

In [53]:
df.shape

(21702, 3)

In [54]:
df.head()

,NIS5,DATE,CASES
8695,35013,2020-03-02,1
9938,41063,2020-03-02,1
2937,21004,2020-03-02,1
3638,21019,2020-03-02,1
4393,23062,2020-03-02,1


### Merge with geolocation